In [200]:
import os
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, LSTM, SimpleRNN
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [201]:
import editdistance

In [202]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [203]:
import tensorflow as tf

In [204]:
from tqdm import tqdm

# 1. Подготовка данных

In [826]:
with open('data/ru.txt', 'r') as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines if not '(' in line]
    lines = lines[9:]

In [831]:
words_pad = []
words_len = []
spells_pad = []
spells_len = []
for line in tqdm(lines):
    chunks = line.split()
    graphs = list(chunks[0])
    phones = chunks[1:]
    if len(graphs) > 8 or len(phones) > 8:
        continue
    words_pad.append(graphs)
    spells_pad.append(phones)
    spells_len.append(len(phones))

100%|██████████| 533911/533911 [00:02<00:00, 256196.90it/s]


In [835]:
words_pad = pad_sequences(words_pad, value='#', maxlen=8, dtype=object)

In [836]:
spells_pad = pad_sequences(spells_pad, value='#', maxlen=8, dtype=object)

In [839]:
spells_len[5]

8

In [840]:
spells_pad[5]

array(['a1', 'a0', 'l', 't', 'o0', 'nj', 'e0', 'n'], dtype=object)

In [841]:
with open('data/phonemes.txt', 'r') as f:
    lines_1 = f.readlines()
    phonemes_dict = [line.strip() for line in lines_1]

In [842]:
with open('data/graphemes.txt', 'r') as f:
    lines_2 = f.readlines()
    graphemes_dict = [line.strip() for line in lines_2]

In [843]:
def one_hot_encoding_symbol(x, dictionary):
    vector = [0]*len(dictionary)
    if x == '#':
        return vector
    else:
        vector[dictionary.index(x)] = 1
    return vector

In [844]:
def one_hot_encoding_symbol_without_masking(x, dictionary):
    vector = [0]*len(dictionary)
    vector[dictionary.index(x)] = 1
    return vector

In [845]:
def get_X_Y(words_pad, spells_pad, graphemes_dict, phonemes_dict):
    words_one_hot = []
    for word in tqdm(words_pad):
        word_one_hot = []
        for grapheme in word:
            word_one_hot.append(one_hot_encoding_symbol_without_masking(grapheme, graphemes_dict))
        words_one_hot.append(word_one_hot)
    
    spells_one_hot = []
    for spell in tqdm(spells_pad):
        spell_one_hot = []
        for phoneme in spell:
            spell_one_hot.append(one_hot_encoding_symbol_without_masking(phoneme, phonemes_dict))
        spells_one_hot.append(spell_one_hot)
    return words_one_hot, spells_one_hot

In [846]:
X, Y = get_X_Y(words_pad, spells_pad, graphemes_dict, phonemes_dict)

100%|██████████| 253403/253403 [00:05<00:00, 43777.37it/s]


In [848]:
X = [np.array(matrix) for matrix in X]
X = [np.expand_dims(matrix, axis=0) for matrix in X]
X = np.row_stack((X))

In [849]:
Y = [np.array(matrix) for matrix in Y]
Y = [np.expand_dims(matrix, axis=0) for matrix in Y]
Y = np.row_stack((Y))

In [850]:
Y.shape

(253403, 8, 49)

In [851]:
X.shape

(253403, 8, 35)

In [880]:
X_tensor = tf.constant(X)

In [882]:
Y_tensor = tf.constant(Y)

In [886]:
spells_len_tensor = tf.cast(tf.constant(spells_len), tf.dtypes.int64)

In [891]:
ds_size = len(spells_len)

In [888]:
ds

<TensorDataset shapes: ((253403, 8, 35), (253403, 8, 49), (253403,)), types: (tf.int64, tf.int64, tf.int64)>

In [859]:
ds = tf.data.Dataset.from_tensor_slices((tf.constant([1, 2]), tf.constant([3, 4])))

In [898]:
for i, j, k in ds:
    print(i, j, k)

tf.Tensor(
[[[0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  ...
  [1 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [1 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  ...
  [1 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  ...
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  ...
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  ...
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]], shape=(253403, 8, 35), dtype=int64) tf.Tensor(
[[[0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]
  ...
  [1 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 1 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 1]
  [0 0 0 ... 0 0 1]

In [854]:
np.save('features.npy', X)

In [855]:
np.save('labels.npy', Y)

In [926]:
int(0.6 * 3)

1

In [962]:
def train_test_split_self(X, Y, Z, train_size=0.7, val_size=0.3):
    np.random.shuffle(X)
    X_train = X[:int(len(X)*train_size)]
    X_test = X[int(len(X)*train_size):]
    np.random.shuffle(X_train)
    X_val = X_train[:int(len(X_train)*val_size)]
    X_train = X_train[int(len(X_train)*val_size):]
    
    np.random.shuffle(Y)
    Y_train = Y[:int(len(Y)*train_size)]
    Y_test = Y[int(len(Y)*train_size):]
    np.random.shuffle(Y_train)
    Y_val = Y_train[:int(len(Y_train)*val_size)]
    Y_train = Y_train[int(len(Y_train)*val_size):]
    
    np.random.shuffle(spells_len)
    Z_train = Z[:int(len(Z)*train_size)]
    Z_test = Z[int(len(Z)*train_size):]
    np.random.shuffle(X_train)
    Z_val = Z_train[:int(len(Z_train)*val_size)]
    Z_train = Z_train[int(len(Z_train)*val_size):]
    
    return X_train, X_test, X_val, Y_train, Y_test, Y_val, Z_train, Z_test, Z_val

In [963]:
X_train, X_test, X_val, Y_train, Y_test, Y_val, Z_train, Z_test, Z_val = train_test_split_self(X, Y, spells_len)

In [863]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=27)

In [941]:
x_train.shape

(177382, 8, 35)

In [967]:
X_test.shape

(76021, 8, 35)

In [968]:
X_train = tf.constant(X_train)
X_test = tf.constant(X_test)
X_val = tf.constant(X_val)
Y_train = tf.constant(Y_train)
Y_test = tf.constant(Y_test)
Y_val = tf.constant(Y_val)
Z_train = tf.constant(Z_train)
Z_test = tf.constant(Z_test)
Z_val = tf.constant(Z_val)

In [969]:
ds = tf.data.Dataset.from_tensors((X_train, Y_train, Z_train))

### Маскирование

In [933]:
# нужно если не использовать дополнительный класс для паддинга 

In [25]:
masked_x_train = masking(x_train)

In [26]:
masked_x_train._keras_mask

<tf.Tensor: shape=(42116, 8), dtype=bool, numpy=
array([[False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       ...,
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True]])>

In [27]:
masked_x_train.shape

TensorShape([42116, 8, 35])

#### разбиваем данные на мини батчи

In [ ]:
test_indices = np.random.choice

In [638]:
y_train.shape

(42116, 8, 49)

In [704]:
model = Sequential()

In [706]:
tf.keras.Model

tensorflow.python.keras.engine.training.Model

In [705]:
model

## Create custom model

In [973]:
class CustomModel(tf.keras.Model):
    def train_step(self, data):
        x, y= data
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [939]:
model = CustomModel()

# 2. Создание модели

In [943]:
inputs = Input((8, 35))

In [944]:
rnn = tf.keras.layers.SimpleRNN(128, activation='tanh', return_sequences=True, input_shape=(8, 35))

In [945]:
x = rnn(inputs)

In [946]:
dense = Dense(49, activation='softmax')

In [947]:
outputs = dense(x)

In [974]:
model_f = CustomModel(inputs=inputs, outputs=outputs)

In [975]:
model_f.summary()

Model: "custom_model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 8, 35)]           0         
_________________________________________________________________
simple_rnn_17 (SimpleRNN)    (None, 8, 128)            20992     
_________________________________________________________________
dense_12 (Dense)             (None, 8, 49)             6321      
Total params: 27,313
Trainable params: 27,313
Non-trainable params: 0
_________________________________________________________________


In [800]:
model = Sequential()
# model.add(tf.keras.layers.Masking(mask_value=0, input_shape=(8, 35)))
model.add(SimpleRNN(, activation='tanh', return_sequences=True, input_shape=(8, 35)))
model.add(Dense(49, activation='softmax'))
# model.add(Dense(48, activation='softmax'))

In [731]:
model = Sequential()
model.add(LSTM(512, input_shape=(8, 35)))

In [801]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_12 (SimpleRNN)    (None, 8, 256)            74752     
_________________________________________________________________
dense_8 (Dense)              (None, 8, 49)             12593     
Total params: 87,345
Trainable params: 87,345
Non-trainable params: 0
_________________________________________________________________


In [787]:
a = tf.constant([[[0, 0, 1], [0, 1, 0]],
                 [[0, 0, 1], [0, 1, 0]]])

In [788]:
a

<tf.Tensor: shape=(2, 2, 3), dtype=int32, numpy=
array([[[0, 0, 1],
        [0, 1, 0]],

       [[0, 0, 1],
        [0, 1, 0]]], dtype=int32)>

In [790]:
tf.argmax(a, axis=2)

<tf.Tensor: shape=(2, 2), dtype=int64, numpy=
array([[2, 1],
       [2, 1]])>

In [683]:
tf.ones(shape=None)

ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.

In [748]:
def dense_to_sparse(dense, pad_value=-1):
    pad_value = tf.constant(pad_value, dtype=tf.int32)
    where = tf.not_equal(dense, pad_value)
    indices = tf.where(where)
    values = tf.gather_nd(dense, indices)
    dense_shape = tf.cast(tf.shape(dense), dtype=tf.int64)
    return tf.SparseTensor(indices, values, dense_shape)

In [797]:
def my_metric_fn(y_true, y_pred):
    input_shape = K.shape(y_pred)
    input_length = tf.ones(shape=input_shape[0]) * K.cast(input_shape[1], 'float32')
    labels_dec, _ = K.ctc_decode(y_pred, input_length, greedy=True)
    hypothesis = dense_to_sparse(tf.cast(labels_dec[0], tf.int32))
    y_true = tf.argmax(y_true, axis=2)
    truth = dense_to_sparse(tf.cast(y_true, dtype=tf.int32))
    curler = tf.edit_distance(hypothesis=hypothesis, truth=truth)
    ler = tf.reduce_mean(curler)
    return ler

In [737]:
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics='accuracy')

In [738]:
early_stopping=EarlyStopping(monitor='val_loss', patience=3)

In [976]:
model_f.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[my_metric_fn, 'accuracy'])

In [977]:
histoty = model_f.fit(x_train, y_train, batch_size=16, epochs=10, verbose=1, callbacks=[early_stopping])

Epoch 1/10
11087/11087 [==============================] - 48s 4ms/step - loss: 0.8423 - my_metric_fn: 0.4255 - accuracy: 0.7209
Epoch 2/10
11087/11087 [==============================] - 48s 4ms/step - loss: 0.8151 - my_metric_fn: 0.4235 - accuracy: 0.7229
Epoch 3/10
11087/11087 [==============================] - 48s 4ms/step - loss: 0.8052 - my_metric_fn: 0.4218 - accuracy: 0.7247
Epoch 4/10
11087/11087 [==============================] - 47s 4ms/step - loss: 0.7989 - my_metric_fn: 0.4203 - accuracy: 0.7260
Epoch 5/10
11087/11087 [==============================] - 48s 4ms/step - loss: 0.7942 - my_metric_fn: 0.4193 - accuracy: 0.7269
Epoch 6/10
11087/11087 [==============================] - 48s 4ms/step - loss: 0.7906 - my_metric_fn: 0.4184 - accuracy: 0.7279
Epoch 7/10
11087/11087 [==============================] - 48s 4ms/step - loss: 0.7879 - my_metric_fn: 0.4175 - accuracy: 0.7286
Epoch 8/10
11087/11087 [==============================] - 48s 4ms/step - loss: 0.7854 - my_metric_fn: 0.

In [971]:
histoty = model_f.fit(ds, batch_size=16, epochs=10, verbose=1, callbacks=[early_stopping],
                     validation_data=(X_val, Y_val))

Epoch 1/10
1/1 [==============================] - 8s 8s/step - loss: 4.5927 - my_metric_fn: 0.9379 - accuracy: 0.1332 - val_loss: 4.4140 - val_my_metric_fn: 0.9375 - val_accuracy: 0.1334
Epoch 2/10
1/1 [==============================] - 7s 7s/step - loss: 4.4190 - my_metric_fn: 0.9374 - accuracy: 0.1330 - val_loss: 4.2568 - val_my_metric_fn: 0.9371 - val_accuracy: 0.1333
Epoch 3/10
1/1 [==============================] - 7s 7s/step - loss: 4.2621 - my_metric_fn: 0.9369 - accuracy: 0.1329 - val_loss: 4.1163 - val_my_metric_fn: 0.9361 - val_accuracy: 0.1334
Epoch 4/10
1/1 [==============================] - 7s 7s/step - loss: 4.1219 - my_metric_fn: 0.9361 - accuracy: 0.1330 - val_loss: 3.9832 - val_my_metric_fn: 0.9350 - val_accuracy: 0.1337
Epoch 5/10
1/1 [==============================] - 7s 7s/step - loss: 3.9890 - my_metric_fn: 0.9347 - accuracy: 0.1333 - val_loss: 3.8593 - val_my_metric_fn: 0.9325 - val_accuracy: 0.1365
Epoch 6/10
1/1 [==============================] - 7s 7s/step - lo

In [802]:
histoty = model.fit(x_train, y_train, batch_size=32, epochs=10, 
                    verbose=1, validation_split=0.3, callbacks=[early_stopping])

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [ ]:
@tf.function
def create_tensor(array):
    return tf.constant(array)

In [120]:
tf.matmul()

TypeError: matmul() missing 2 required positional arguments: 'a' and 'b'

In [96]:
a = create_tensor([1, 2, 3])

In [113]:
type(a)

tensorflow.python.framework.ops.EagerTensor

In [102]:
b = tf.constant([1, 2, 4])

In [103]:
b

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 4], dtype=int32)>

In [100]:
a

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 2, 3], dtype=int32)>

In [ ]:
with sess.as

In [290]:
histoty = model.fit(x_train, y_train, batch_size=32, epochs=10, 
                    verbose=1, validation_split=0.3, callbacks=[early_stopping])

Epoch 1/10
922/922 [==============================] - 4s 4ms/step - loss: 1.2991 - accuracy: 0.3607 - val_loss: 1.3024 - val_accuracy: 0.3582
Epoch 2/10
922/922 [==============================] - 4s 4ms/step - loss: 1.2974 - accuracy: 0.3594 - val_loss: 1.3012 - val_accuracy: 0.3618
Epoch 3/10
922/922 [==============================] - 4s 4ms/step - loss: 1.2962 - accuracy: 0.3598 - val_loss: 1.3001 - val_accuracy: 0.3565
Epoch 4/10
922/922 [==============================] - 4s 4ms/step - loss: 1.2954 - accuracy: 0.3590 - val_loss: 1.2993 - val_accuracy: 0.3571
Epoch 5/10
922/922 [==============================] - 4s 4ms/step - loss: 1.2946 - accuracy: 0.3586 - val_loss: 1.2991 - val_accuracy: 0.3602
Epoch 6/10
922/922 [==============================] - 4s 4ms/step - loss: 1.2941 - accuracy: 0.3586 - val_loss: 1.2986 - val_accuracy: 0.3578
Epoch 7/10
170/922 [====>.........................] - ETA: 2s - loss: 1.2998 - accuracy: 0.3558

KeyboardInterrupt: 

In [82]:
def get_vector_word(word, dictionary):
    vector = []
    letters = pad_sequences([list(word)], value='#', maxlen=8, dtype=object)
    for letter in letters[0]:
        letter_vector = one_hot_encoding_symbol(letter, dictionary)
        vector.append(letter_vector)
    return vector

In [83]:
vector_word = get_vector_word('мама', graphemes_dict)

In [84]:
a = np.array(vector_word)

In [85]:
x = np.expand_dims(a, axis=0)

In [86]:
x.shape

(1, 8, 35)

In [87]:
y = model.predict(x)

In [100]:
for vector_phoneme in y[0]:
    print(phonemes_dict[vector_phoneme.argmax()])

sch
vj
y0
n
a1
t
t
t


In [337]:
def my_metric_fn(y_true, y_pred):
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)

In [340]:
my_metric_fn(np.array([1, 2]), np.array([1, 2]))

<tf.Tensor: shape=(), dtype=int64, numpy=0>

In [332]:
y_true = np.array([4, 6])
y_pred = np.array([2, 3])
squared_difference = tf.square(y_true - y_pred)

In [333]:
squared_difference

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([4, 9])>

In [336]:
tf.reduce_mean(squared_difference, axis=-1).numpy()

6

In [329]:
m = tf.keras.metrics.AUC()
# m.update_state([0, 1, 1, 1], [0, 1, 0, 0])
# print('Intermediate result:', float(m.result()))

In [326]:
m.reset_states()

In [328]:
m.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [330]:
m.update_state

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [ ]:
tf.sparse.SparseTensor()

In [344]:
import editdistance

In [85]:
pip3 uninstall te

/usr/local/lib/python3.6/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.2.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


ImportError: cannot import name 'keras_tensor'

In [378]:
def my_metric_fn(y_true, y_pred):
    array_true = y_true.numpy()
    print(array_true.shape)
    edit_dist = editdistance.eval(y_true, y_pred)
    ler = round(edit_dist / len(y_true) * 100, 1)
    return ler

In [377]:
my_metric_fn(list('мама'), list('мома'))

['a0', 'a1', 'b', 'bj', 'v', 'vj', 'g', 'gj', 'd', 'dj', 'zh', 'z', 'zj', 'i0', 'i1', 'j', 'k', 'kj', 'l', 'lj', 'm', 'mj', 'n', 'nj', 'o0', 'o1', 'p', 'pj', 'r', 'rj', 's', 'sj', 't', 'tj', 'u0', 'u1', 'f', 'fj', 'h', 'hj', 'c', 'ch', 'sh', 'sch', 'y0', 'y1', 'e0', 'e1', '#']


25.0

In [363]:
a = tf.Variable(5, name="a")
b = tf.Variable(7, name="b")

In [370]:
c = (b**2 - a**3)**5
print(c)

tf.Tensor(1759441920, shape=(), dtype=int32)


In [385]:
c.numpy()

1759441920

In [390]:
a = tf.Tensor()

TypeError: __init__() missing 3 required positional arguments: 'op', 'value_index', and 'dtype'

In [86]:
import tf.keras.backand as K

ModuleNotFoundError: No module named 'tf'

In [375]:
phonemes_dict

['a0',
 'a1',
 'b',
 'bj',
 'v',
 'vj',
 'g',
 'gj',
 'd',
 'dj',
 'zh',
 'z',
 'zj',
 'i0',
 'i1',
 'j',
 'k',
 'kj',
 'l',
 'lj',
 'm',
 'mj',
 'n',
 'nj',
 'o0',
 'o1',
 'p',
 'pj',
 'r',
 'rj',
 's',
 'sj',
 't',
 'tj',
 'u0',
 'u1',
 'f',
 'fj',
 'h',
 'hj',
 'c',
 'ch',
 'sh',
 'sch',
 'y0',
 'y1',
 'e0',
 'e1',
 '#']

In [ ]:
edit_dist = editdistance.eval(y_true, y_pred)

In [83]:
h = tf.Tensor()

TypeError: __init__() missing 3 required positional arguments: 'op', 'value_index', and 'dtype'

In [90]:
K.

<tf.Tensor: shape=(), dtype=bool, numpy=True>

In [521]:
hypothesis = tf.sparse.SparseTensor(
    [[0, 0, 0],
     [1, 0, 0]],
    ["a", "b"],
    (2, 1, 1))

In [528]:
hypothesis.shape

TensorShape([2, 1, 1])

In [ ]:
def create_sparse(ten):
    n = len(ten)
    ind = [[xi, 0, yi] for xi,x in enumerate(ten) for yi,y in enumerate(x)]
    chars = list(''.join(ten))
    return K.tf.SparseTensorValue(ind, chars, [n,1,1])

In [645]:
for xi, x in enumerate([1, 2, 3]):
    print(xi)
    print(x)

0
1
1
2
2
3


In [ ]:
print

In [ ]:
obj1 = tf.SparseTensor(indices=[[0, ], []])

In [668]:
hypothesis = tf.constant([[2, 0, 0], [0, 1, 0], [0, 0, 1]])

In [677]:
true = tf.constant([[1, 0, 0], [1, 0, 0], [0, 0, 1]])

In [670]:
hypothesis_sparse = tf.sparse.from_dense(hypothesis)

In [671]:
true_sparse = tf.sparse.from_dense(true)

In [672]:
hypothesis_sparse.indices

<tf.Tensor: shape=(3, 2), dtype=int64, numpy=
array([[0, 0],
       [1, 1],
       [2, 2]])>

In [673]:
true_sparse.indices

<tf.Tensor: shape=(3, 2), dtype=int64, numpy=
array([[0, 0],
       [1, 0],
       [2, 2]])>

In [674]:
hypothesis_sparse.values

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([2, 1, 1], dtype=int32)>

In [675]:
true_sparse.values

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([1, 1, 1], dtype=int32)>

In [676]:
tf.edit_distance(hypothesis_sparse, true_sparse, normalize=True)

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([1., 0., 0.], dtype=float32)>

In [678]:
a = tf.sparse.SparseTensor(indices=[[0, 0], [1, 1], [2, 2]], values=[1, 1, 1], dense_shape=(3, 2))

In [681]:
a.indices

<tf.Tensor: shape=(3, 2), dtype=int64, numpy=
array([[0, 0],
       [1, 1],
       [2, 2]])>